# Devoir 5, Question 2 : Sélection de variables

# Homework 5, Question 2: Feature selection

## Code préambule

## Preamble code

In [ ]:
import time
import numpy
import warnings
import os
import requests
import urllib.parse
import pickle as pkl
import zipfile
import io
import pandas

# Nous ne voulons pas avoir ce type d'avertissement, qui
# n'est pas utile dans le cadre de ce devoir
# We do not want to have this type of warning, which
# is not useful in the context of this homework
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, RFE, chi2, mutual_info_classif

# Fonction pour charger le jeu de données CSDMC Spam
# Function to load the CSDMC Spam dataset
def load_csdmc_spam():
    csdmdir = "/pax/shared/GIF-4101-7005/csdmc-spam"
    X = numpy.load(os.path.join(csdmdir, "X.npy"))
    y = numpy.load(os.path.join(csdmdir, "y.npy"))
    with open(os.path.join(csdmdir, "features.pkl"), "rb") as f:
        features = pkl.load(f)
    return X, y, features

# Fonctions utilitaires liées à l'évaluation
# Utility functions related to evaluation
_times = []
def checkTime(maxduration, question):
    duration = _times[-1] - _times[-2]
    if duration > maxduration:
        print("[ATTENTION] Votre code pour la question {0} met trop de temps à s'exécuter! ".format(question)+
            "Le temps maximum permis est de {0:.4f} secondes, mais votre code a requis {1:.4f} secondes! ".format(maxduration,duration)+
            "Assurez-vous que vous ne faites pas d'appels bloquants (par exemple à show()) dans cette boucle!") 

# Définition des durées d'exécution maximales pour chaque sous-question
# Definition of maximum execution times for each sub-question
TMAX_Q2Aall = 0.5
TMAX_Q2Achi = 0.5
TMAX_Q2Amut = 60
TMAX_Q2B = 150

Soit le jeu de données *CSDMC 2010 SPAM corpus*, dont nous avons extrait un [jeu de données](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/csdmc-spam.zip) avec 1000 caractéristiques binaires, indiquant la présence ou non d’un mot particulier. 

Information sur le jeu de données:
- `X` : ensemble de données, chaque ligne étant une instance et chaque colonne une caractéristique;
- `y` : étiquettes de classe des données, 0 indiquant que l’instance est un pourriel, 1 indiquant que c’est un message légitime;
- `features` : la liste des 1000 mots d’intérêt, dans le même ordre que les colonnes de `X`.

Supposons que l’on veut sélectionner les variables d’intérêt de ce jeu de données, où chaque variable indique la présence ou non d’un mot particulier.

Consider the *CSDMC 2010 SPAM corpus* dataset, from which we have extracted a [dataset](https://pax.ulaval.ca/static/GIF-4101-7005/fichiers/csdmc-spam.zip) with 1000 binary characteristics, indicating whether or not a particular word is present.

Dataset information:
- `X` : dataset, each row being an instance and each column a characteristic;
- `y` : class labels of the data, 0 indicating that the instance is spam, 1 indicating that it is a legitimate message;
- `features` : the list of 1000 words of interest, in the same order as the columns of `X`.

Suppose we want to select the variables of interest from this dataset, where each variable indicates whether or not a particular word is present.

## Q2A
Effectuez une sélection univariée en conservant les 10 meilleures variables parmi les 1000, selon le test du $\chi^2$ (`feature_selection.chi2`) et le critère d’information mutuelle (`feature_selection.mutual_info_classif`).

Les données sont partitionnées en un jeu d'entraînement et jeu de test, de même taille. Effectuez la sélection de variables selon le jeu d’entraînement. Rapportez les variables (et les mots correspondants) sélectionnées par les deux critères de sélection utilisés. Rapportez également la performance de classement sur le jeu de test, en utilisant un SVM linéaire (`svm.LinearSVC`) comme modèle de classement entraîné sur la partition d’entraînement. Comparez les résultats avec ceux obtenus avec la même configuration, mais utilisant l’ensemble des 1000 variables.

## Q2A
Make a univariate selection keeping the 10 best variables among the 1000, according to the $\chi^2$ test (`feature _selection.chi2`) and the mutual information criterion (`feature_ selection.mutual _info_ classif`).

The data is partitioned into a training set and a testing set, of equal size. Perform the variable selection according to the training set. Report the variables (and corresponding words) selected by the two selection criteria used. Also report the classification performance on the testing set, using a linear SVM (`svm.LinearSVC`) as the trained classification model on the training partition. Compare the results with those obtained with the same setup, but using all 1000 variables.

### Entrez votre solution à Q2A dans la cellule suivante, en vous basant sur le patron de code donné plus bas.

### Enter your answer to Q2A in the following cell, starting from the code template given below.

In [ ]:
# Chargement des données et des noms des caractéristiques
# Loading data and feature names
X, y, features_ = load_csdmc_spam()

# Division du jeu en entraînement / test, ne modifiez pas la valeur de random_state
# Split the game into training / test, do not change the value of random_state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données *complet*
# et rapportez sa performance en test dans *score* et les caractéristiques dans *features*
# Train a linear SVM classifier on the *complete* dataset and report its test performance
# in *score* and its features in *features*
clf = LinearSVC()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
features = numpy.unique(features_)
# ******

print("Variable:", features)
print("Score avec toutes les variables: ", score)
_times.append(time.time())
checkTime(TMAX_Q2Aall, "2A (toutes les variables)")

_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données
# en réduisant le nombre de caractéristiques (features) à 10 en
# utilisant le chi^2 comme métrique et rapportez sa performance en test.
# Train a linear SVM classifier on the dataset
# by reducing the number of features to 10 by
# using chi^2 as a metric and report its test performance.
clf = LinearSVC()

kbest_chi2 = SelectKBest(chi2, k=10).fit(X_train, y_train)
X_train_chi2 = kbest_chi2.transform(X_train)
X_test_chi2  = kbest_chi2.transform(X_test)

clf.fit(X_train_chi2, y_train)
score = clf.score(X_test_chi2, y_test)

features = [features_[i] for i in numpy.where(kbest_chi2.get_support() == True)[0]]
# ******

print("Variables:", features)
print("Score avec variables sélectionnées par chi2: ", score)
_times.append(time.time())
checkTime(TMAX_Q2Achi, "2A (sélection de variables par chi2)")

_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données
# en réduisant le nombre de caractéristiques (features) à 10 en utilisant
# l'information mutuelle comme métrique et rapportez sa performance en test.
# Train a linear SVM classifier on the dataset by reducing the number of
# features to 10 using mutual information as a metric and report its 
# test performance.
clf = LinearSVC()

kbest_info = SelectKBest(mutual_info_classif, k=10).fit(X_train, y_train)
X_train_info = kbest_info.transform(X_train)
X_test_info = kbest_info.transform(X_test)

clf.fit(X_train_info, y_train)

score = clf.score(X_test_info, y_test)

features = [features_[i] for i in numpy.where(kbest_info.get_support() == True)[0]]
# ******

print("Variables: ", features)
print("Score avec variables sélectionnées par information mutuelle: ", score)
_times.append(time.time())
checkTime(TMAX_Q2Amut, "2A (sélection de variables par information mutuelle)")

### Patron de code réponse à l'exercice Q2A

### Q2A answer code template

In [ ]:
# Chargement des données et des noms des caractéristiques
# Loading data and feature names
X, y, features = load_csdmc_spam()

# Division du jeu en entraînement / test, ne modifiez pas la valeur de random_state
# Split the game into training / test, do not change the value of random_state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données *complet*
# et rapportez sa performance en test dans *score* et les caractéristiques dans *features*
# Train a linear SVM classifier on the *complete* dataset and report its test performance
# in *score* and its features in *features*
# ******

print("Variable:", features)
print("Score avec toutes les variables: ", score)
_times.append(time.time())
checkTime(TMAX_Q2Aall, "2A (toutes les variables)")

_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données
# en réduisant le nombre de caractéristiques (features) à 10 en
# utilisant le chi^2 comme métrique et rapportez sa performance en test.
# Train a linear SVM classifier on the dataset
# by reducing the number of features to 10 by
# using chi^2 as a metric and report its test performance.
# ******

print("Variables:", features)
print("Score avec variables sélectionnées par chi2: ", score)
_times.append(time.time())
checkTime(TMAX_Q2Achi, "2A (sélection de variables par chi2)")

_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données
# en réduisant le nombre de caractéristiques (features) à 10 en utilisant
# l'information mutuelle comme métrique et rapportez sa performance en test.
# Train a linear SVM classifier on the dataset by reducing the number of
# features to 10 using mutual information as a metric and report its 
# test performance.
# ******

print("Variables: ", features)
print("Score avec variables sélectionnées par information mutuelle: ", score)
_times.append(time.time())
checkTime(TMAX_Q2Amut, "2A (sélection de variables par information mutuelle)")

## Q2B
Répétez les expérimentations de la sous-question précédente avec la sélection séquentielle arrière implémentée dans la fonction `feature_selection.RFE` de scikit-learn. Faites les expérimentations en utilisant un SVM linéaire comme modèle de base, en faisant un réentraînement à chaque itération de l’algorithme (`step=1`).

## Q2B
Repeat the experiments of the previous sub-question with the backward sequential selection implemented in the `feature_selection.RFE` function of scikit-learn. Make the experiments using a linear SVM as a basic model, retraining at each iteration of the algorithm (`step=1`).

### Entrez votre solution à Q2B dans la cellule suivante, en vous basant sur le patron de code donné plus bas.

### Enter your answer to Q2B in the following cell, starting from the code template given below.

In [ ]:
_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données
# en réduisant le nombre de caractéristiques (features) à 10 par
# sélection séquentielle arrière et rapportez sa performance en test.
# Train a linear SVM classifier on the dataset
# by reducing the number of features to 10 using backward sequential
# selection as a metric and report its test performance.
clf = LinearSVC()
rfe = RFE(clf, n_features_to_select=10)

rfe.fit(X_train, y_train)
score = rfe.score(X_test, y_test)

features = [features_[i] for i in numpy.where(rfe.get_support() == True)[0]]
# ******

print("Variables: ", features)
print("Score avec sélection arrière séquentielle (RFE):", score)
_times.append(time.time())
checkTime(TMAX_Q2B, "2B")

### Patron de code réponse à l'exercice Q2B

### Q2B answer code template

In [ ]:
_times.append(time.time())
score = 0
features = []

# *** TODO ***
# Entraînez un classifieur SVM linéaire sur le jeu de données
# en réduisant le nombre de caractéristiques (features) à 10 par
# sélection séquentielle arrière et rapportez sa performance en test.
# Train a linear SVM classifier on the dataset
# by reducing the number of features to 10 using backward sequential
# selection as a metric and report its test performance.
# ******

print("Variables: ", features)
print("Score avec sélection arrière séquentielle (RFE):", score)
_times.append(time.time())
checkTime(TMAX_Q2B, "2B")

## Q2C
En guise d'analyse, répondez aux questions suivantes sur la sélection de variables:
1. Comparez les résultats obtenus par les différentes approches de sélection de variables présentées dans les sous-questions précédentes et expliquez les différences observées.
2. Expliquez pourquoi un algorithme de sélection arrière séquentielle est souvent préféré à un algorithme de sélection avant séquentielle pour des données comportant des dépendances non-linéaires complexes entre les variables.

## Q2C
For analysis, answer the following questions about feature selection:
1. Compare the results obtained by the different feature selection methods presented in the previous sub-questions and explain the observed differences.
2. Explain why a sequential backward selection algorithm is often preferred over a sequential forward selection algorithm for data with complex nonlinear dependencies between variables.

### Entrez votre solution à Q2C dans la cellule ci-dessous
Patron de réponse en format markdown:
~~~md
1. Comparez les résultats obtenus par les différentes approches de sélection de variables présentées dans les sous-questions précédentes et expliquez les différences observées.
***Donnez ici vos explications au point 1***

2. Expliquez pourquoi un algorithme de sélection arrière séquentielle est souvent préféré à un algorithme de sélection avant séquentielle pour des données comportant des dépendances non-linéaires complexes entre les variables.
***Donnez ici vos explications au point 2***
~~~

### Enter your solution to Q2C in the cell below
Markdown answer template:
~~~md
1. Compare the results obtained by the different feature selection approaches presented in the previous sub-questions and explain the observed differences.
***Give here your explanations to point 1***

2. Explain why a sequential backward selection algorithm is often preferred over a sequential forward selection algorithm for data with complex nonlinear dependencies between variables.
***Give here your explanations to point 2***
~~~

1. Comparez les résultats obtenus par les différentes approches de sélection de variables présentées dans les sous-questions précédentes et expliquez les différences observées. / Compare the results obtained by the different feature selection approaches presented in the previous sub-questions and explain the observed differences.
***Donnez ici vos explications au point 1 / Give here your explanations to point 1***

2. Expliquez pourquoi un algorithme de sélection arrière séquentielle est souvent préféré à un algorithme de sélection avant séquentielle pour des données comportant des dépendances non-linéaires complexes entre les variables. / Explain why a sequential backward selection algorithm is often preferred over a sequential forward selection algorithm for data with complex nonlinear dependencies between variables.
***Donnez ici vos explications au point 2 / Give here your explanations to point 2***